In [60]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import KFold, GridSearchCV

from sklearn import svm
df_svm = pd.read_csv('svm-data.csv', names=['a', 'b','c'],)

X = df_svm.drop(['a'], axis=1)
y = df_svm['a']

svc = svm.SVC(kernel='linear', C=100000, random_state=241)
svc.fit(X, y)

print (svc.support_ + 1)
#print (df_svm)

[ 4  5 10]


In [25]:
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space'])
print (newsgroups.target)

[0 0 1 ..., 1 1 0]


In [27]:
from sklearn.feature_extraction.text import  TfidfVectorizer
tf_idf = TfidfVectorizer()

X = tf_idf.fit_transform(newsgroups.data)
y = (newsgroups.target)

feature_mapping = tf_idf.get_feature_names()



In [31]:
# Словарик с поисковыми параметрами С
grid = {'C': np.power(10.0, np.arange(-5, 6))}
print (grid)

# кросс валидация по 5 блокам
cv = KFold(n_splits=5, shuffle=True, random_state=241)
# обучение на support vector machine
clf = svm.SVC(kernel='linear', random_state=241)

# посик наилучшего параметра для svm по списку grid 
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)

gs.fit(X, y)


{'C': array([  1.00000000e-05,   1.00000000e-04,   1.00000000e-03,
         1.00000000e-02,   1.00000000e-01,   1.00000000e+00,
         1.00000000e+01,   1.00000000e+02,   1.00000000e+03,
         1.00000000e+04,   1.00000000e+05])}
1
1


In [34]:
#оценка качетсва по кросс-валидации и параметр С 
for a in gs.grid_scores_:
    print (a.mean_validation_score)
    print (a.parameters)
    # a.mean_validation_score — оценка качества по кросс-валидации
    # a.parameters — значения параметров


0.552631578947
{'C': 1.0000000000000001e-05}
0.552631578947
{'C': 0.0001}
0.552631578947
{'C': 0.001}
0.552631578947
{'C': 0.01}
0.950167973124
{'C': 0.10000000000000001}
0.993281075028
{'C': 1.0}
0.993281075028
{'C': 10.0}
0.993281075028
{'C': 100.0}
0.993281075028
{'C': 1000.0}
0.993281075028
{'C': 10000.0}
0.993281075028
{'C': 100000.0}


C:\Sneg\Add.Program\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [37]:
#Обучение при лучшем парамтер C

С = gs.best_params_
clf_best = svm.SVC(kernel='linear', random_state=241, C=C)
clf_best.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=241, shrinking=True,
  tol=0.001, verbose=False)

In [119]:
arr = np.array(clf_best.coef_.data)

arr = np.sort(np.fabs(arr))
arr[:] = arr[::-1]
top_arr = arr[:10]
print (top_arr)

[ 2.66316479  1.9203794   1.25468995  1.24918001  1.20161118  1.1801316
  1.13908084  1.13061234  1.09709365  1.02930693]


In [204]:
# создать dataFrame с колонками 'index' & 'data' и вставить в них индекс и дата (по модулю из обученной модели)
i=0
df = pd.DataFrame(columns=['index', 'data'])
while i < len(clf_best.coef_.indices):
    #print (clf_best.coef_.indices)
    df1 = pd.DataFrame([[clf_best.coef_.indices[i], math.fabs(clf_best.coef_.data[i])]], columns=['index', 'data'])
    df = df.append(df1, ignore_index=True)
    i+=1  

In [226]:
# отсортировать по столбцу 'data' и потом начать с наибольшего
df_sort = df.sort_values('data')
df_sort = df[::-1]
# содать новые индексы у df
df_new_index = df.reset_index()
print(df_new_index[:10])

a = df_new_index['index'][0]
print (a)

   level_0  index      data
0    16693  24019  2.663165
1    18092  12871  1.920379
2    17449   5088  1.254690
3    17045   5093  1.249180
4     9015  17802  1.201611
5    17536  23673  1.180132
6    16935  21850  1.139081
7    16746   5776  1.130612
8    17422  15606  1.097094
9    17469  22936  1.029307
24019


In [227]:
#создать список с 10 самымы повторяющемися словами и отсортировать его и вы
word = []
i = 0
while i <10:
    a = df_new_index['index'][i]
    print (a)
    word.append(feature_mapping[a])
    i+=1
print (np.sort(word))

24019
12871
5088
5093
17802
23673
21850
5776
15606
22936
['atheism' 'atheists' 'bible' 'god' 'keith' 'moon' 'religion' 'sci' 'sky'
 'space']
